###  Shyam bhaskar (Roll Number:S20210010035)
### Yalakanti Eswar (Roll Number:S20210020332)

In [49]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import hvplot.pandas
from scipy.stats import boxcox
from sklearn.svm import SVR
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.diagnostic import het_white
import tensorflow as tf
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense,Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
df = pd.read_csv("PRSA_data_2010.1.1-2014.12.31.csv")
# df.columns
df

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43819,43820,2014,12,31,19,8.0,-23,-2.0,1034.0,NW,231.97,0,0
43820,43821,2014,12,31,20,10.0,-22,-3.0,1034.0,NW,237.78,0,0
43821,43822,2014,12,31,21,10.0,-22,-3.0,1034.0,NW,242.70,0,0
43822,43823,2014,12,31,22,8.0,-22,-4.0,1034.0,NW,246.72,0,0


In [6]:

df.head()

,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0


In [7]:
df.dropna(inplace=True)

In [8]:
df.shape

(41757, 12)

In [9]:
df.head()

,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
24,2010,1,2,0,129.0,-16,-4.0,1020.0,SE,1.79,0,0
25,2010,1,2,1,148.0,-15,-4.0,1020.0,SE,2.68,0,0
26,2010,1,2,2,159.0,-11,-5.0,1021.0,SE,3.57,0,0
27,2010,1,2,3,181.0,-7,-5.0,1022.0,SE,5.36,1,0
28,2010,1,2,4,138.0,-7,-5.0,1022.0,SE,6.25,2,0


In [10]:
hourly_data = df.groupby('hour')['pm2.5'].mean()
hourly_data

hour
0     113.390202
1     113.698567
2     110.323174
3     108.042980
4     104.080275
5     100.013218
6      96.882759
7      96.024727
8      95.907940
9      94.672800
10     93.447520
11     91.678633
12     89.448873
13     87.990280
14     86.283247
15     85.534215
16     85.922280
17     87.571183
18     91.507710
19     97.398393
20    104.614368
21    109.249856
22    111.021252
23    111.889782
Name: pm2.5, dtype: float64

In [9]:
# df.groupby('day').mean()

In [11]:
df['cbwd'] = df['cbwd'].astype(str)

In [12]:
encoded_cbwd = pd.get_dummies(df['cbwd'], prefix='cbwd')

# Concatenate the encoded columns with the original DataFrame
df_encoded = pd.concat([df, encoded_cbwd], axis=1)

# Drop the original 'cbwd' column as it's no longer needed
df_encoded.drop('cbwd', axis=1, inplace=True)

In [13]:
X = df_encoded.drop(columns=['pm2.5'])
y = (np.round(np.log(df_encoded['pm2.5'] + 0.01),3)).astype('float')
min(y)

-4.605

## MODEL

### ANN with SVR

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [41]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
x_sc_tr=X_train1
x_sc_te=X_test1
x_sc_tr.iloc[:,:-4] = scaler.fit_transform(X_train1.iloc[:,:-4])
x_sc_te.iloc[:,:-4] = scaler.transform(X_test1.iloc[:,:-4])

In [42]:
ann_model = Sequential([
        Dense(256, activation='tanh', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.2), 
    Dense(128, activation='tanh', input_shape=(X_train_scaled.shape[1],)),
    BatchNormalization(),
    Dropout(0.2),          
    Dense(64, activation='tanh'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='tanh'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(16, activation='tanh'),
    BatchNormalization(),
    Dense(12) 
])


ann_model.compile(optimizer='adam', loss='mse')

checkpoint_path = "model_checkpoint.h5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
ann_model.fit(x_sc_tr, y_train1, epochs=100, batch_size=32, validation_split=0.2,callbacks=[checkpoint])


Epoch 1/100
824/836 [============================>.] - ETA: 0s - loss: 5.0587
Epoch 1: val_loss improved from inf to 0.62186, saving model to model_checkpoint.h5
836/836 [==============================] - 7s 5ms/step - loss: 5.0006 - val_loss: 0.6219
Epoch 2/100
832/836 [============================>.] - ETA: 0s - loss: 0.6488
Epoch 2: val_loss improved from 0.62186 to 0.54708, saving model to model_checkpoint.h5
836/836 [==============================] - 4s 5ms/step - loss: 0.6489 - val_loss: 0.5471
Epoch 3/100
829/836 [============================>.] - ETA: 0s - loss: 0.6054
Epoch 3: val_loss improved from 0.54708 to 0.52543, saving model to model_checkpoint.h5
836/836 [==============================] - 4s 5ms/step - loss: 0.6048 - val_loss: 0.5254
Epoch 4/100
824/836 [============================>.] - ETA: 0s - loss: 0.5739
Epoch 4: val_loss improved from 0.52543 to 0.49554, saving model to model_checkpoint.h5
836/836 [==============================] - 4s 5ms/step - loss: 0.5731 - v

836/836 [==============================] - 4s 5ms/step - loss: 0.4227 - val_loss: 0.3336
Epoch 35/100
828/836 [============================>.] - ETA: 0s - loss: 0.4143
Epoch 35: val_loss did not improve from 0.33356
836/836 [==============================] - 4s 5ms/step - loss: 0.4153 - val_loss: 0.3386
Epoch 36/100
829/836 [============================>.] - ETA: 0s - loss: 0.4167
Epoch 36: val_loss did not improve from 0.33356
836/836 [==============================] - 4s 5ms/step - loss: 0.4170 - val_loss: 0.3451
Epoch 37/100
833/836 [============================>.] - ETA: 0s - loss: 0.4131
Epoch 37: val_loss improved from 0.33356 to 0.33347, saving model to model_checkpoint.h5
836/836 [==============================] - 4s 5ms/step - loss: 0.4126 - val_loss: 0.3335
Epoch 38/100
825/836 [============================>.] - ETA: 0s - loss: 0.4145
Epoch 38: val_loss improved from 0.33347 to 0.32952, saving model to model_checkpoint.h5
836/836 [==============================] - 4s 5ms/step

In [45]:
loade_model = load_model(checkpoint_path)
extracted_features_train = loade_model.predict(x_sc_tr)
extracted_features_test = loade_model.predict(x_sc_te)

scaler = StandardScaler()
scaled_extracted_features_train = scaler.fit_transform(extracted_features_train)
scaled_extracted_features_test = scaler.transform(extracted_features_test)


svr_model = SVR(C=10, kernel='rbf')
svr_model.fit(scaled_extracted_features_train, y_train1)

y_pred = svr_model.predict(scaled_extracted_features_test)


261/261 [==============================] - 1s 2ms/step


In [48]:
mse = mean_squared_error(y_test, y_pred)
r2_scores=r2_score(y_test,y_pred)
print('The MSE VALUE:',mean_absolute_error(y_test,y_pred))
print("The MAE :",mean_absolute_error(y_test,y_pred))
print('the R-squared value:',r2_scores)

0.4060975920108758
0.7259723402106982


### Linear Regression

In [14]:

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Model Training
model = LinearRegression()
model.fit(X_train, y_train)

# Model Evaluation
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("The MAE :",mean_absolute_error(y_test,y_pred))
print("R^2 Score:", r2)

Mean Squared Error: 0.6142367499177182
R^2 Score: 0.42277517048720126


### SVR

In [ ]:
model_svr = SVR(C=10, kernel='rbf')
model_svr.fit(X_train_scaled, y_train)
y_pred_svr = model_svr.predict(X_test_scaled)
mse_svr = mean_squared_error(y_test, y_pred_svr)
r2_svr = r2_score(y_test, y_pred_svr)

print("\nSupport Vector Regression:")
print("Mean Squared Error:", mse_svr)
print("The MAE :",mean_absolute_error(y_test,y_pred))
print("R^2 Score:", r2_svr)


### Polynomial 

In [ ]:
poly = PolynomialFeatures(degree=3)  # You can adjust the degree as needed
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

model_plr = LinearRegression()
model_plr.fit(X_train_poly, y_train)
y_pred_plr = model_plr.predict(X_test_poly)
mse_plr = mean_squared_error(y_test, y_pred_plr)
r2_plr = r2_score(y_test, y_pred_plr)

print("\nPolynomial Regression:")
print("Mean Squared Error:", mse_plr)
print("The MAE :",mean_absolute_error(y_test,y_pred))
print("R^2 Score:", r2_plr)